# Faulty Steel Plates

In [1]:
%pip install tqdm torch scikit-learn numpy pandas matplotlib pytest
%pip install ../../PiShield

%reload_ext autoreload
%autoreload 2


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Processing /Users/arnavkohli/Work/Uni Stuff/FYP/PiShield
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pishield: filename=pishield-1.0.0-py3-none-any.whl size=46556 sha256=825d5665c54608876fd126afc091c45c21bcde7792c569bf14abb56e8400f242
  Stored in directory: /private/var/folders/xs/_tk2cd552r7b4t8tnjrwjj800000gn/T/pip-ephem-wheel-cache-f3wjb7s6/wheels/c4/b7/f0/be8df562ae22530e49ca9acd400e4e886eef8cdb0803eb8d21
Successfully built pishield
  Attempting uninstall: pishield
    Found existing installation: pishield 1.0.0
    Uninstalling pishield-1.0.0:
      Successfully uninstalled pishield-1.0.0

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader
from sklearn.compose import ColumnTransformer
from tqdm import tqdm
import logging

In [3]:
logging.basicConfig(level=logging.WARNING)

In [4]:
devices = ["cuda", "mps"]
device = "cpu"

for possible_device in devices:
    if possible_device == "cuda" and torch.cuda.is_available():
        device = "cuda"
        break
    elif possible_device == "mps" and torch.backends.mps.is_available():
        device = "mps"
        break

device = torch.device(device)

print(f"Using Device: {device}")

Using Device: mps


In [5]:
# seed_value = 42

# random.seed(seed_value)
# np.random.seed(seed_value)
# torch.manual_seed(seed_value)

# if torch.cuda.is_available():
#     torch.cuda.manual_seed_all(seed_value)
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = False

In [6]:
fsp_dataset = pd.read_csv("faults.csv")
display(fsp_dataset)

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,42,50,270900,270944,267,17,44,24220,76,108,...,0.8182,-0.2913,0.5822,1,0,0,0,0,0,0
1,645,651,2538079,2538108,108,10,30,11397,84,123,...,0.7931,-0.1756,0.2984,1,0,0,0,0,0,0
2,829,835,1553913,1553931,71,8,19,7972,99,125,...,0.6667,-0.1228,0.2150,1,0,0,0,0,0,0
3,853,860,369370,369415,176,13,45,18996,99,126,...,0.8444,-0.1568,0.5212,1,0,0,0,0,0,0
4,1289,1306,498078,498335,2409,60,260,246930,37,126,...,0.9338,-0.1992,1.0000,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936,249,277,325780,325796,273,54,22,35033,119,141,...,-0.4286,0.0026,0.7254,0,0,0,0,0,0,1
1937,144,175,340581,340598,287,44,24,34599,112,133,...,-0.4516,-0.0582,0.8173,0,0,0,0,0,0,1
1938,145,174,386779,386794,292,40,22,37572,120,140,...,-0.4828,0.0052,0.7079,0,0,0,0,0,0,1
1939,137,170,422497,422528,419,97,47,52715,117,140,...,-0.0606,-0.0171,0.9919,0,0,0,0,0,0,1


In [7]:
from pishield.linear_requirements.parser import parse_constraints_file, remap_constraint_variables

constraints_file = "constraints.txt"
ordering, constraints = parse_constraints_file(constraints_file)
ordering, constraints, reverse_mapping = remap_constraint_variables(ordering, constraints)
num_variables = len(ordering)
print(f"Number of variables: {num_variables}, Number of constraints: {len(constraints)}")

target_columns = list(map(lambda var: fsp_dataset.columns[int(var.split('_')[1])], reverse_mapping.values()))

Number of variables: 7, Number of constraints: 4


In [8]:
from pishield.linear_requirements.utils import check_constraint_satisfaction

fsp_dataset_shuffled = fsp_dataset.sample(frac=1, random_state=42).reset_index(drop=True)

target_columns = list(map(lambda var: fsp_dataset_shuffled.columns[int(var.split('_')[1])], reverse_mapping.values()))
all_column_names = fsp_dataset_shuffled.columns.tolist()
input_columns = [col for col in all_column_names if col not in target_columns]

X = fsp_dataset_shuffled[input_columns]
y = fsp_dataset_shuffled[target_columns]

continuous_input_features = [
    'Pixels_Areas', 'X_Perimeter', 'Y_Perimeter',
    'Length_of_Conveyer', 'Steel_Plate_Thickness', 'Edges_Index',
    'Empty_Index', 'Square_Index', 'Outside_X_Index', 'Edges_X_Index',
    'Edges_Y_Index', 'Outside_Global_Index', 'LogOfAreas', 'Log_X_Index',
    'Log_Y_Index', 'Orientation_Index', 'Luminosity_Index', 'SigmoidOfAreas'
]

categorical_input_features = [
    'TypeOfSteel_A300', 'TypeOfSteel_A400',
    'Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults'
]

assert set(continuous_input_features + categorical_input_features) == set(input_columns)

# Split the dataset into 70-10-20
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.125, random_state=42
)

In [9]:
# X_train, y_train = torch.tensor(X_train.to_numpy(), dtype=torch.float32), torch.tensor(y_train.to_numpy(), dtype=torch.float32)
# X_val, y_val = torch.tensor(X_val.to_numpy(), dtype=torch.float32), torch.tensor(y_val.to_numpy(), dtype=torch.float32)
# X_test, y_test = torch.tensor(X_test.to_numpy(), dtype=torch.float32), torch.tensor(y_test.to_numpy(), dtype=torch.float32)

In [10]:
# Changes the column order, first columns are the scaled continuous columns,
# the following columns are categorical (binary in this case) and are passed
# through
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), continuous_input_features),
        ('cat', 'passthrough', categorical_input_features)
    ],
)

X_train = torch.tensor(preprocessor.fit_transform(X_train), dtype=torch.float32)
X_val = torch.tensor(preprocessor.transform(X_val), dtype=torch.float32)
X_test = torch.tensor(preprocessor.transform(X_test), dtype=torch.float32)

y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)

print(f"Constraints Satisfied: {check_constraint_satisfaction(y_train, constraints) \
        and check_constraint_satisfaction(y_val, constraints) \
            and check_constraint_satisfaction(y_test, constraints)}")

Constraints Satisfied: True


In [11]:
# Disabling shufle for a little more determinism in measuring
# performance across architectures
train_loader = DataLoader(
    TensorDataset(X_train, y_train), batch_size=32, shuffle=True
)

val_loader = DataLoader(
    TensorDataset(X_val, y_val), batch_size=32
)

test_loader = DataLoader(
    TensorDataset(X_test, y_test), batch_size=32
)

In [12]:
from pishield.linear_requirements.utils import check_constraint_satisfaction
from pishield.linear_requirements.adjusted_constraint_loss import adjusted_constraint_loss
import torch
import torch.nn.functional as F

def train_and_evaluate(model, optimizer, train_loader, val_loader,
                       num_epochs=10, optim_layer=False):
    # Move model to device
    model.to(device)

    for epoch in range(num_epochs):
        # Training
        model.train()
        epoch_train_loss = 0.0
        progress_bar = tqdm(train_loader, leave=False)
        progress_bar.set_description(f"Epoch {epoch+1}/{num_epochs}")

        training_runs, runs_satisfied = 0, 0

        for i, (inputs, targets) in enumerate(progress_bar):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()

            if optim_layer:
                outputs = model(inputs, targets)
                masks = model.shield_layer.masks
                loss = adjusted_constraint_loss(outputs, targets, masks)
            else:
                outputs = model(inputs)
                loss = F.mse_loss(outputs, targets)

            loss.backward()
            optimizer.step()
            epoch_train_loss += loss.item()
            training_runs += 1
            runs_satisfied += int(check_constraint_satisfaction(targets, constraints))
            if i % 10 == 0:
                progress_bar.set_postfix(loss=loss.item())

        print(f"Constraints satisfied (training): {runs_satisfied}/{training_runs}")
        avg_train_loss = epoch_train_loss / len(train_loader)

        if optim_layer and hasattr(model, 'shield_layer'):
            if hasattr(model.shield_layer, 'temperature'):
                current_temp = model.shield_layer.temperature.item()
                model.shield_layer.anneal_temperature(device)
                new_temp = model.shield_layer.temperature.item()
                if epoch % 1 == 0:
                    print(f"Annealed temperature from {current_temp:.4f} to {new_temp:.4f}")

        # Evaluation
        model.eval()
        with torch.no_grad():
            val_predictions = []
            val_targets_list = []
            validation_runs, runs_satisfied = 0, 0

            for val_inputs, val_targets in val_loader:
                val_inputs = val_inputs.to(device)
                val_targets = val_targets.to(device)
                val_outputs = model(val_inputs)

                # Store outputs and targets (move to CPU for later processing)
                val_predictions.append(val_outputs.cpu())
                val_targets_list.append(val_targets.cpu())

                validation_runs += 1
                runs_satisfied += int(check_constraint_satisfaction(val_outputs, constraints))

            # Concatenate all batches
            all_val_predictions = torch.cat(val_predictions, dim=0)
            all_val_targets = torch.cat(val_targets_list, dim=0)

            val_rmse = torch.sqrt(F.mse_loss(all_val_predictions, all_val_targets))
            print(f"Constraints satisfied (validation): {runs_satisfied}/{validation_runs}")

        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Val RMSE: {val_rmse:.4f}")

In [13]:
def test_model(model, test_loader, constraints):
    model.eval()
    model.to(device)

    test_predictions = []
    test_targets_list = []
    test_runs, runs_satisfied = 0, 0

    with torch.no_grad():
        for test_inputs, test_targets in test_loader:
            test_inputs = test_inputs.to(device)
            test_targets = test_targets.to(device)
            test_outputs = model(test_inputs)

            # Store outputs and targets (move to CPU for later processing)
            test_predictions.append(test_outputs.cpu())
            test_targets_list.append(test_targets.cpu())

            test_runs += 1
            runs_satisfied += int(check_constraint_satisfaction(test_outputs, constraints))

    # Concatenate all batches
    all_test_predictions = torch.cat(test_predictions, dim=0)
    all_test_targets = torch.cat(test_targets_list, dim=0)

    test_rmse = torch.sqrt(F.mse_loss(all_test_predictions, all_test_targets))

    print(f"\nConstraints satisfied (testing): {runs_satisfied}/{test_runs}")
    print(f"Test RMSE: {test_rmse:.4f}")

In [14]:
class BasicMLPNet(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, output_size)
        )

    def forward(self, x):
        return self.layers(x)

model = BasicMLPNet(len(input_columns), len(target_columns))
optimizer = optim.Adam(model.parameters())

train_and_evaluate(
    model, optimizer, train_loader,
    val_loader, num_epochs=10,
    optim_layer=False
)

test_model(model, test_loader, constraints)

Constraints satisfied (training): 43/43
Constraints satisfied (validation): 0/7
Epoch 1/10 | Train Loss: 1602889521866.4187 | Val RMSE: 1476069.1250


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 0/7
Epoch 2/10 | Train Loss: 1661289682515.3489 | Val RMSE: 1476066.7500


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 0/7
Epoch 3/10 | Train Loss: 1618346582968.5581 | Val RMSE: 1476056.8750


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 0/7
Epoch 4/10 | Train Loss: 1614884549274.7908 | Val RMSE: 1476031.1250


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 0/7
Epoch 5/10 | Train Loss: 1607350669407.2559 | Val RMSE: 1475980.6250


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 0/7
Epoch 6/10 | Train Loss: 1615318979798.3257 | Val RMSE: 1475895.6250


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 0/7
Epoch 7/10 | Train Loss: 1612241258353.1162 | Val RMSE: 1475766.1250


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 0/7
Epoch 8/10 | Train Loss: 1630243793324.6511 | Val RMSE: 1475585.8750


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 0/7
Epoch 9/10 | Train Loss: 1601627019597.3953 | Val RMSE: 1475348.1250


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 0/7
Epoch 10/10 | Train Loss: 1602194716052.8372 | Val RMSE: 1475043.8750

Constraints satisfied (testing): 0/13
Test RMSE: 1359953.3750


In [15]:
from pishield.linear_requirements.shield_layer import ShieldLayer

class ShieldMLPNet(nn.Module):
    def __init__(self, input_size, output_size, constraints_file):
        super().__init__()
        self.mlp = BasicMLPNet(input_size, output_size)
        self.shield_layer = ShieldLayer(output_size, constraints_file)

    def forward(self, x, y=None):
        out = self.shield_layer(self.mlp(x), y)
        return out

In [16]:
original_model = ShieldMLPNet(len(input_columns), len(target_columns), constraints_file)

optimizer = optim.Adam(original_model.parameters())

train_and_evaluate(
    original_model, optimizer, train_loader,
    val_loader, num_epochs=10,
    optim_layer=False
)

test_model(original_model, test_loader, constraints)

All constraints
 + y_1 - y_0 > 0.0
 + y_3 - y_2 > 0.0
 + y_6 - y_5 > 0.0
 + y_4 - y_6 > 0.0


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 7/7
Epoch 1/10 | Train Loss: 1637785061995.1628 | Val RMSE: 1476069.0000


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 7/7
Epoch 2/10 | Train Loss: 1601971841071.6279 | Val RMSE: 1476066.1250


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 7/7
Epoch 3/10 | Train Loss: 1614477585955.7209 | Val RMSE: 1476053.8750


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 7/7
Epoch 4/10 | Train Loss: 1616728618007.8140 | Val RMSE: 1476023.2500


...

Constraints satisfied (training): 43/43
Constraints satisfied (validation): 7/7
Epoch 6/10 | Train Loss: 1616001329961.6743 | Val RMSE: 1475868.1250


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 7/7
Epoch 7/10 | Train Loss: 1622233878242.2327 | Val RMSE: 1475726.5000


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 7/7
Epoch 8/10 | Train Loss: 1602843456154.7908 | Val RMSE: 1475528.5000


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 7/7
Epoch 9/10 | Train Loss: 1609812085736.1860 | Val RMSE: 1475269.3750


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 7/7
Epoch 10/10 | Train Loss: 1624398869146.7908 | Val RMSE: 1474937.0000

Constraints satisfied (testing): 13/13
Test RMSE: 1359851.6250


In [17]:
improved_model = ShieldMLPNet(len(input_columns), len(target_columns), constraints_file)

optimizer = optim.Adam(improved_model.parameters())

train_and_evaluate(
    improved_model, optimizer, train_loader,
    val_loader, num_epochs=10,
    optim_layer=True
)

test_model(improved_model, test_loader, constraints)

All constraints
 + y_1 - y_0 > 0.0
 + y_3 - y_2 > 0.0
 + y_6 - y_5 > 0.0
 + y_4 - y_6 > 0.0


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 7/7
Epoch 1/10 | Train Loss: 1641006429207.8140 | Val RMSE: 1476068.8750


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 7/7
Epoch 2/10 | Train Loss: 1616622394820.4651 | Val RMSE: 1476065.8750


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 7/7
Epoch 3/10 | Train Loss: 1603930535316.8372 | Val RMSE: 1476054.8750


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 7/7
Epoch 4/10 | Train Loss: 1612997755642.0466 | Val RMSE: 1476029.1250


...

Constraints satisfied (training): 43/43
Constraints satisfied (validation): 7/7
Epoch 6/10 | Train Loss: 1606246657571.7209 | Val RMSE: 1475899.6250


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 7/7
Epoch 7/10 | Train Loss: 1642033151237.9534 | Val RMSE: 1475781.6250


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 7/7
Epoch 8/10 | Train Loss: 1603877860375.8140 | Val RMSE: 1475618.0000


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 7/7
Epoch 9/10 | Train Loss: 1601932921832.1860 | Val RMSE: 1475401.8750


Constraints satisfied (training): 43/43
Constraints satisfied (validation): 7/7
Epoch 10/10 | Train Loss: 1612657485871.6279 | Val RMSE: 1475133.6250

Constraints satisfied (testing): 13/13
Test RMSE: 1359978.7500
